In [2]:
import numpy as np, pandas as pd
import glob,os,sys,argparse,shutil,gc,copy,warnings,random,logging,multiprocessing,time
from datetime import timedelta
from wsi import slide,filters,tiles,util

START_TIME = time.time()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()


In [3]:
# DATA_DIR = '/home/jovyan/data'
DATA_DIR = os.environ['DIR_RAW_DATA']

df = pd.read_csv(f'{DATA_DIR}/train.csv',sep=',')

df['slide_id'] = df.filename.str.split('.').str[0]
df['og_bbox'] = df.apply(lambda x: [x.x1,x.y1,x.x2,x.y2] ,axis=1)
df['corr_bbox'] = df.apply(lambda x: [x.x1,x.y1,min(x.max_x,x.x2),min(x.max_y,x.y2)] ,axis=1)

df['w'] = df.x2-df.x1
df['h'] = df.y2-df.y1
df['xpad'] = df.max_x - df.x2
df['ypad'] = df.max_y - df.y2

bads = ['7HxL729fl6_b_38839_78455_39604_79279', 'HpWI7vJms2_a_71182_57113_71902_57462', 'i9xm71KbYG_b_31467_113976_31986_114541', 'JvxiXClFKl_a_20316_9218_20451_9502', 'JvxiXClFKl_a_69751_37180_70029_37225', 'Lzx7XfUujk_a_13308_34726_13336_34737', 'Lzx7XfUujk_b_10468_84122_10792_84991', 'Lzx7XfUujk_b_15486_145827_15738_146587', 'Lzx7XfUujk_b_17181_145345_17593_145910', 'Lzx7XfUujk_b_53455_82217_54007_82559', 'Lzx7XfUujk_b_63207_142583_63569_143562', 'Lzx7XfUujk_b_64715_142167_65022_142754', 'Lzx7XfUujk_b_67782_116477_68140_117554', 'Lzx7XfUujk_b_8802_84908_9241_85324', 'rzsagNFXMn_a_15458_148084_15486_148204', 'TFrBjcO8nJ_b_63103_127967_63818_128712', 'WipCgQtJPE_b_25648_118162_26120_118743', 'yJxYpOCh6m_b_18928_24013_18941_24038']
df['uniqid'] = df.slide_id + '_'+ df.og_bbox.apply(lambda x: '_'.join([str(i) for i in x]))
df_bads = df[df.uniqid.isin(bads)].reset_index(drop=True)
df = df[~df.uniqid.isin(bads)].reset_index(drop=True)
print(f'dropped {len(df_bads)}',len(df))

dropped 18 909


In [4]:
meta = df.drop_duplicates(subset='filename').reset_index(drop=True)
meta.shape

(247, 15)

In [10]:
##Generate masks
NAMES = meta.slide_id.tolist()#[:10]

print(f'##### MASKING {len(NAMES)} SLIDES #####')

filters.multiprocess_apply_filters_to_images(image_name_list=NAMES,)
elapsed = time.time() - START_TIME
print(f'##### TIME MASKING SLIDES: {timedelta(seconds=elapsed)} #####')


##### MASKING 247 SLIDES #####
Applying filters to images (multiprocess)

Number of processes: 5 Number of training images: 247
Time to apply filters to all images (multiprocess): 0:02:19.100617

##### TIME MASKING SLIDES: 0:02:19.562460 #####
